In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

In [3]:
# this code takes all words in patterns along with puntuncation and stop words and add to the words list 
# and also adds each pattern along with respective tag to 'documents'
# and all types of tag into one list called 'classes'

words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.','(',')','[',']','&',"'s",':']
intents_file = open('intents.json').read()
intents = json.loads(intents_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((pattern, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
print(documents)

[('Hi there', 'greeting'), ('How are you', 'greeting'), ('Is anyone there?', 'greeting'), ('Hey', 'greeting'), ('Hola', 'greeting'), ('Hello', 'greeting'), ('Good day', 'greeting'), ('Bye', 'goodbye'), ('See you later', 'goodbye'), ('Goodbye', 'goodbye'), ('Nice chatting to you, bye', 'goodbye'), ('Till next time', 'goodbye'), ('Thanks', 'thanks'), ('Thank you', 'thanks'), ("That's helpful", 'thanks'), ('Awesome, thanks', 'thanks'), ('Thanks for helping me', 'thanks'), ('How you could help me?', 'options'), ('What you can do?', 'options'), ('What help you provide?', 'options'), ('How you can be helpful?', 'options'), ('What support is offered', 'options'), ('what is the difference between covariance and correlation?', '199'), ('difference between covariance and correlation?', '199'), ('what is correlation and covariance?', '199'), ('What is a LogLoss function and where is it used?', '200'), ('what is LogLoss function?', '200'), ('What do you mean by Cross Entropy?', '201'), ('Given Dec

In [5]:
print(classes)

['greeting', 'goodbye', 'thanks', 'options', '199', '200', '201', '202', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '242', '243', '244', '245', '246', '249', '253', '254', '255', '256', '260', '261', '262', '264']


In [6]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]

In [7]:
words

['hi',
 'there',
 'how',
 'are',
 'you',
 'is',
 'anyone',
 'there',
 'hey',
 'hola',
 'hello',
 'good',
 'day',
 'bye',
 'see',
 'you',
 'later',
 'goodbye',
 'nice',
 'chatting',
 'to',
 'you',
 'bye',
 'till',
 'next',
 'time',
 'thanks',
 'thank',
 'you',
 'that',
 'helpful',
 'awesome',
 'thanks',
 'thanks',
 'for',
 'helping',
 'me',
 'how',
 'you',
 'could',
 'help',
 'me',
 'what',
 'you',
 'can',
 'do',
 'what',
 'help',
 'you',
 'provide',
 'how',
 'you',
 'can',
 'be',
 'helpful',
 'what',
 'support',
 'is',
 'offered',
 'what',
 'is',
 'the',
 'difference',
 'between',
 'covariance',
 'and',
 'correlation',
 'difference',
 'between',
 'covariance',
 'and',
 'correlation',
 'what',
 'is',
 'correlation',
 'and',
 'covariance',
 'what',
 'is',
 'a',
 'logloss',
 'function',
 'and',
 'where',
 'is',
 'it',
 'used',
 'what',
 'is',
 'logloss',
 'function',
 'what',
 'do',
 'you',
 'mean',
 'by',
 'cross',
 'entropy',
 'given',
 'decision',
 'tree',
 'random',
 'forest',
 'which

In [8]:
#collecting root words(lemmatization) from 'words' list and saving after sorting 
# sorted 'classes' (nothing but tags in data) and saving


# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

77 documents
56 classes ['199', '200', '201', '202', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '242', '243', '244', '245', '246', '249', '253', '254', '255', '256', '260', '261', '262', '264', 'goodbye', 'greeting', 'options', 'thanks']
231 unique lemmatized words ['+1', '-1', '2', 'a', 'a/b', 'acceptable', 'accuracy', 'adopts', 'advantage', 'affinity', 'algorithm', 'alpha', 'an', 'analysis', 'and', 'ann', 'anyone', 'anything', 'are', 'at', 'aware', 'awesome', 'bagging', 'based', 'be', 'before', 'between', 'bias', 'black', 'box', 'building', 'by', 'bye', 'called', 'can', 'case', 'categorical', 'cause', 'chatting', 'class', 'classification', 'coefficient', 'collaborative', 'collecting', 'component', 'confidence', 'confounding', 'connection', 'considered', 'continuous', 'correct', 'correlation',

In [9]:
#saving words and classes as pickle format to load later
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [10]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words=[]
    word = nltk.word_tokenize(doc[0])
    pattern_words.extend(word)
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
     
    #print(bag)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [11]:
training

[[[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [12]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

C:\Users\Pavan786\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
training.shape

(77, 2)

In [13]:
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [14]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
16/16 [==============================] - 0s 3ms/step - loss: 4.0936 - accuracy: 0.0390
Epoch 2/200
16/16 [==============================] - 0s 2ms/step - loss: 3.9875 - accuracy: 0.0390
Epoch 3/200
16/16 [==============================] - 0s 2ms/step - loss: 3.9246 - accuracy: 0.1299
Epoch 4/200
16/16 [==============================] - 0s 2ms/step - loss: 3.8343 - accuracy: 0.1169
Epoch 5/200
16/16 [==============================] - 0s 3ms/step - loss: 3.7150 - accuracy: 0.1299
Epoch 6/200
16/16 [==============================] - 0s 3ms/step - loss: 3.6896 - accuracy: 0.1299
Epoch 7/200
16/16 [==============================] - 0s 2ms/step - loss: 3.5205 - accuracy: 0.1429
Epoch 8/200
16/16 [==============================] - 0s 2ms/step - loss: 3.4968 - accuracy: 0.1429
Epoch 9/200
16/16 [==============================] - 0s 2ms/step - loss: 3.3962 - accuracy: 0.2078
Epoch 10/200
16/16 [==============================] - 0s 2ms/step - loss: 3.1683 - accuracy: 0.2727
Epoch 11/

16/16 [==============================] - 0s 2ms/step - loss: 0.6687 - accuracy: 0.7922
Epoch 84/200
16/16 [==============================] - 0s 2ms/step - loss: 0.4752 - accuracy: 0.8701
Epoch 85/200
16/16 [==============================] - 0s 2ms/step - loss: 0.5503 - accuracy: 0.8182
Epoch 86/200
16/16 [==============================] - 0s 2ms/step - loss: 0.3763 - accuracy: 0.8831
Epoch 87/200
16/16 [==============================] - 0s 2ms/step - loss: 0.5602 - accuracy: 0.8182
Epoch 88/200
16/16 [==============================] - 0s 2ms/step - loss: 0.5071 - accuracy: 0.8571
Epoch 89/200
16/16 [==============================] - 0s 2ms/step - loss: 0.3931 - accuracy: 0.8831
Epoch 90/200
16/16 [==============================] - 0s 2ms/step - loss: 0.3909 - accuracy: 0.8831
Epoch 91/200
16/16 [==============================] - 0s 1ms/step - loss: 0.2866 - accuracy: 0.9091
Epoch 92/200
16/16 [==============================] - 0s 1ms/step - loss: 0.4160 - accuracy: 0.8831
Epoch 93/200


Epoch 164/200
16/16 [==============================] - 0s 3ms/step - loss: 0.3086 - accuracy: 0.9221
Epoch 165/200
16/16 [==============================] - 0s 2ms/step - loss: 0.4012 - accuracy: 0.8961
Epoch 166/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2056 - accuracy: 0.9221
Epoch 167/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2942 - accuracy: 0.8831
Epoch 168/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2703 - accuracy: 0.9091
Epoch 169/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2521 - accuracy: 0.9091
Epoch 170/200
16/16 [==============================] - 0s 2ms/step - loss: 0.4427 - accuracy: 0.8831
Epoch 171/200
16/16 [==============================] - 0s 2ms/step - loss: 0.2525 - accuracy: 0.9091
Epoch 172/200
16/16 [==============================] - 0s 2ms/step - loss: 0.3178 - accuracy: 0.8831
Epoch 173/200
16/16 [==============================] - 0s 2ms/step - loss: 0.3900 - accurac

In [16]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))


def TF_IDF(sentence):
    # bag of words - vocabulary matrix
    vectors=vectorizer.transform([sentence])
    dense = vectors.todense()
    li=dense.tolist()
    return(li[0])


def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    #print(tag)
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [17]:
res=predict_class('Hi there')
res

[{'intent': 'greeting', 'probability': '0.9999963'}]

In [20]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")

while(flag==True):
    user_response = input()
    if (user_response!='bye'):
            if (greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                ints = predict_class(user_response)
                if (len(ints)==0):
                 answer='Sorry I am still learning'
                else:
                 answer=getResponse(ints, intents)
                print("ROBO: ",end="")
                print(answer)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
hi
ROBO: hello
Entropy?
ROBO: Good to see you again
